# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


C:\Users\fcama\Anaconda2\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables relative to whether someone got admitted or not. Think in terms of for a certain prestige level, how many people got admitted and didnt get admitted

In [4]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df['admit'],df['prestige'],rownames=['admit'], margins=True)

prestige,1.0,2.0,3.0,4.0,All
admit,,,,,
0,28,95,93,55,271
1,33,53,28,12,126
All,61,148,121,67,397


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [5]:
# Dummifying prestige valiables
dummy_prestige = pd.get_dummies(df["prestige"], prefix="prestige")
dummy_prestige.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We need n-1 or 3 dummy variables in this case, this to avoid multicollinearity.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [6]:
# frequency table for prestige and whether or not someone was admitted
pd.crosstab(df['admit'],df['prestige'],rownames=['admit'], margins=True)

prestige,1.0,2.0,3.0,4.0,All
admit,,,,,
0,28,95,93,55,271
1,33,53,28,12,126
All,61,148,121,67,397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [7]:
# p = outcome desired/all possible outcomes
p_1 = 33/61.0
p_1

0.5409836065573771

In [8]:
# Odds = p / (1 - p)
odds_1 = p_1 / (1 - p_1)
odds_1

1.1785714285714288

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [9]:
# p = outcome desired/all possible outcomes
p_na_1 = 93/336.0
p_na_1

0.2767857142857143

In [10]:
# Odds = p / (1 - p)
odds_na_1 = p_na_1 / (1 - p_na_1)
odds_na_1

0.3827160493827161

#### 3.3 Calculate the odds ratio

In [11]:
# Odds Ratio = odds attended #1 school / odds did not attend #1 school
odds_ratio = odds_1/odds_na_1
odds_ratio

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: 

~ 3.08:1 are the changes or likelyhood for a student's application to be admitted into UCLA who attended a number one prestige school compared to one who did not.

#### 3.5 Print the cross tab for prestige_4

In [67]:
# Print prestige_4 column
prestige_4 = pd.crosstab(data['admit'],data['prestige_4.0'],rownames=['admit'], margins=True)
prestige_4

prestige_4.0,0,1,All
admit,,,
0,216,55,271
1,114,12,126
All,330,67,397


#### 3.6 Calculate the OR 

In [15]:
# p = outcome desired/all possible outcomes
p_4 = 12/67.0
p_4

0.1791044776119403

In [16]:
odds_4 = p_4 / (1 - p_4)
odds_4

0.21818181818181817

In [17]:
# Odds Ratio = odds attended #1 school / odds attended #4 school
OR = odds_1 / odds_4
OR

5.401785714285715

In [18]:
OR = odds_4 / odds_1
OR

0.18512396694214872

#### 3.7 Write this finding in a sentence

Answer: 

~0.19 are the odds or likelyhood of a student attended prestige 4 school admitted into UCLA's program compared to someone attended to prestige 1.   It's easier to understand that the odds of a student attending prestige 1 compare to prestige 4 are 5:1.

## Part 4. Analysis

In [19]:
# create a clean data frame for the regression
cols_to_keep = ["admit","gre","gpa"]
data = df[cols_to_keep].join(dummy_prestige.loc[:,'prestige_2':])
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [20]:
# manually add the intercept
data['intercept'] = 1

#### 4.1 Set the covariates to a variable called train_cols

In [21]:
train_cols = data[data.columns[1:]]
train_cols.head()

,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,380.0,3.61,0,1,0,1
1,660.0,3.67,0,1,0,1
2,800.0,4.00,0,0,0,1
3,640.0,3.19,0,0,1,1
4,520.0,2.93,0,0,1,1


#### 4.2 Fit the model

In [22]:
logit = sm.Logit(data['admit'], train_cols)
# Fit model
result = logit.fit()
result

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [23]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Tue, 21 Nov 2017   Pseudo R-squ.:                 0.08166
Time:                        23:08:05   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa              0.7793      0.333      2.344      0.019       0.128       1.431
prestige_2.0    -0.6801      0.317     -2.146      0.032      -1.301      -0.059
prestige_3.0    -1.3387      0.345     -3.882      0.000      -2.015      -0.663
prestige_4.0    -1.5534      0.417     -3.721      0.000      -2.372      -0.735
intercept       -3.8769      1.142     -3.393      0.001      -6.116      -1.638
================================================================================
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [24]:
params = result.params

In [25]:
# Odds ratio => OR = e ^ (B1)
OR = np.exp(params)
OR

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

In [26]:
conf = result.conf_int()
conf

,0,1
gre,0.000074,0.004362
gpa,0.127619,1.431056
prestige_2.0,-1.301337,-0.058936
prestige_3.0,-2.014579,-0.662776
prestige_4.0,-2.371624,-0.735197
intercept,-6.116077,-1.637631


In [27]:
conf['OR'] = params
conf.columns = ['2.5%', '97.5%','OR']
np.exp(conf)

,2.5%,97.5%,OR
gre,1.000074,1.004372,1.002221
gpa,1.136120,4.183113,2.180027
prestige_2.0,0.272168,0.942767,0.506548
prestige_3.0,0.133377,0.515419,0.262192
prestige_4.0,0.093329,0.479411,0.211525
intercept,0.002207,0.194440,0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: The odds ratio tells us how a unit increase or decrease in the variable affects the odds of being admited. In this case the odds of a student attended to prestige 2 school decrease by 50%

#### 4.6 Interpret the OR of GPA

Answer: Similarly, for every unit change in GPA the odds vary by ~ 2.01 or 201% 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [28]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [29]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [30]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_prestige = pd.get_dummies(combos['prestige'], prefix='prestige')
# dummy_prestige.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']
dummy_prestige.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0


In [31]:
# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa']
combos = combos[cols_to_keep].join(dummy_prestige.loc[:, 'prestige_2.0':])
combos['intercept']=1
combos.head()

,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,220.0,2.260000,0,0,0,1
1,220.0,2.260000,1,0,0,1
2,220.0,2.260000,0,1,0,1
3,220.0,2.260000,0,0,1,1
4,220.0,2.453333,0,0,0,1


In [32]:
train_cols.head()

,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept
0,380.0,3.61,0,1,0,1
1,660.0,3.67,0,1,0,1
2,800.0,4.00,0,0,0,1
3,640.0,3.19,0,0,1,1
4,520.0,2.93,0,0,1,1


#### 5.2 Make predictions on the enumerated dataset

In [33]:
combos['admit_pred'] = result.predict(combos)

In [34]:
combos.head()

,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept,admit_pred
0,220.0,2.260000,0,0,0,1,0.164173
1,220.0,2.260000,1,0,0,1,0.090492
2,220.0,2.260000,0,1,0,1,0.048977
3,220.0,2.260000,0,0,1,1,0.039890
4,220.0,2.453333,0,0,0,1,0.185907


#### 5.3 Interpret findings for the last 4 observations

In [65]:
combos.tail(4)

,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0,intercept,admit_pred
396,800.0,4.0,0,0,0,1,0.734040
397,800.0,4.0,1,0,0,1,0.582995
398,800.0,4.0,0,1,0,1,0.419833
399,800.0,4.0,0,0,1,1,0.368608


Answer: 

Prediction for last four observations:
396 = This student's application, (800-gre, 4.0=pga, and prestige=1),has a high likelyhood that it will be admited into UCLA's master program, ~73.4%
397 = This student's application, (800-gre, 4.0=pga, and prestige=2),has a medium likelyhood that it will be admited into UCLA's master program, ~58.3%
398 = This student's application, (800-gre, 4.0=pga, and prestige=3),has a low likelyhood that it will be admited into UCLA's master program, ~42.0%
399 = This student's application, (800-gre, 4.0=pga, and prestige=4),has a low likelyhood that it will be admited into UCLA's master program, ~36.7%

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [69]:
# isolate gre and class rank
def isolate_and_plot(variable):
    grouped = pd.pivot_table(combos, values=['admit_pred'], index=[variable, 'prestige'], 
                             aggfunc=np.mean)
    
    # in case you're curious as to what this looks like
    # print grouped.head()
    #                      admit_pred
    # gre        prestige            
    # 220.000000 1           0.282462
    #            2           0.169987
    #            3           0.096544
    #            4           0.079859
    # 284.444444 1           0.311718
colors = ['black', 'blue', 'red', 'purple']
for col in combos.dummy_prestige.unique():
    plt_data = grouped.loc[grouped.index.get_level_values(1)==col]
    pl.plot(plt_data.index.get_level_values(0), plt_data['admit_pred'],
                color=colors[int(col)])

    pl.xlabel(variable)
    pl.ylabel("P(admit=1)")
    pl.legend(['1', '2', '3', '4'], loc='upper left', title='Prestige')
    pl.title("Prob(admit=1) isolating " + variable + " and presitge")
    pl.show()

AttributeError: 'DataFrame' object has no attribute 'dummy_prestige'

In [60]:
isolate_and_plot('gre')  

KeyError: 'prestige'

In [56]:
   # make a plot


IndentationError: unexpected indent (<ipython-input-56-55f62f650f33>, line 2)

In [57]:
isolate_and_plot('gpa')

KeyError: 'prestige'